## Custom Fastai Model Inference using PyTorch Base Container

This notebook walks you through setting up FastAI Model you just trained using BYOC for inference using endpoint. First we need to create a Predictor class to accept jpeg images as input and output JSON. The default behaviour is to accept a numpy array.

Update Sagemaker SDK if not already done so

In [11]:
!pip install -U sagemaker

     |████████████████████████████████| 420 kB 29.7 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 20.8 MB/s  eta 0:00:01
     |████████████████████████████████| 86 kB 14.3 MB/s  eta 0:00:01
     |████████████████████████████████| 106 kB 104.7 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 8.5 MB/s s eta 0:00:01
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl (20 kB)
  Created wheel for sagemaker: filename=sagemaker-2.48.1-py2.py3-none-any.whl size=590468 sha256=a7df12212e84a52f2526c0a52b7a52de4188060b351b2f65eaca82be0a414a45
  Stored in directory: /home/ec2-user/.cache/pip/wheels/16/3f/df/2ad2bfd42c4ee903e27b045603175514134c036ac5ef134217
Successfully built sagemaker
  Attempting uninstall: dill
    Found existing installation: dill 0.3.3
    Uninstalling dill-0.3.3:
      Successfully uninstalled dill-0.3.3
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.0
    Uninstalling smdebug-rules

Import necessary libraries

In [4]:
import sagemaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

In [1]:
bucket="your-bucket-name"
prefix="model-prefix-from-your-bucket"

## Update the Image Predictor Class

In the next cell, we will update the predictor class to accept json serializer and deserializer and accept application/x-image content type

In [7]:
jpeg_serializer = sagemaker.serializers.IdentitySerializer("application/x-image")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

### Ineference Script
Create an Inference Script along with any libraries we need installed inside requirements.txt and save them in the inf_src folder

In [2]:
%cat inf_src/serve.py

import logging, requests, os, io, glob, time
from fastai.vision.all import *
from PIL import Image
import json

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

JSON_CONTENT_TYPE = 'application/json'
PNG_CONTENT_TYPE = 'application/x-image'

# loads the model into memory from disk and returns it
def model_fn(model_dir):
    logger.info('model_fn')
    learn = load_learner(os.path.join(model_dir, 'model.pth'))
    return learn

# Deserialize the Invoke request body into an object we can perform prediction on
def input_fn(request_body, content_type=PNG_CONTENT_TYPE):
    logger.info('Deserializing the input data.')
    # process an image uploaded to the endpoint
    # if content_type == PNG_CONTENT_TYPE: return open_image(io.BytesIO(request_body))
    if content_type == PNG_CONTENT_TYPE:
        
        # image_data = Image.open(io.BytesIO(request_body))
        image_data=bytes(request_body)
        return(image_data)
    # process a URL submitted to the endpoint
  

In [3]:
%cat inf_src/requirements.txt

fastai

Provide the location of your Fast AI Model

In [110]:
s3_model_loc=f's3://{bucket}/{prefix}/model.tar.gz'

### Prepare the model
Using the model and ImagePredictor class from above, prepare the model for deployment as an endpoint and provide a serving script that can upack the model

In [111]:
from sagemaker.pytorch import PyTorchModel
from sagemaker.utils import name_from_base

model=PyTorchModel(model_data=s3_model_loc, name=name_from_base("fastai-custom-cont-mod"),
    role=role, framework_version='1.8.0', py_version='py3', entry_point='inf_src/serve.py',source_dir= 'inf_src',predictor_cls=ImagePredictor)

### Deploy the model 

Deploy the model to the end point using ml.m4.xlarge

In [112]:
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------------!

## Create a Predictor to make inference 

In [10]:
from sagemaker.predictor import Predictor
from sagemaker.predictor import json_serializer

predictor=Predictor(endpoint_name='fastai-custom-cont-mod-2021-07-12-01-48-39-278', sagemaker_session=sagemaker_session,serializer=jpeg_serializer)
with open('../Test/Test/Roundabout/R1.png', 'rb') as f:
    img_byte=f.read()
    print(predictor.predict(img_byte))

b'{"class_name": "Roundabout", "confidence": 0.9967899918556213}'


### Using Boto3 Library

In [11]:
import boto3
client=boto3.client('sagemaker-runtime')
im_name="/home/ec2-user/SageMaker/manyFrameworks/Test/Test/Roundabout/R2.png"

response = client.invoke_endpoint(
EndpointName='fastai-custom-cont-mod-2021-07-12-01-48-39-278',
ContentType='application/x-image',
Body=open(im_name, 'rb').read())

In [12]:
import json
json.loads(response['Body'].read().decode("utf-8"))

{'class_name': 'Roundabout', 'confidence': 0.9481703042984009}